# 3.1章　LLMに知識を与える

## 3.1.1 LLM に知識を与える

In [ ]:
!pip install langchain
!pip install langchain-openai

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

model = ChatOpenAI(model="gpt-4o-mini")
result = model.invoke([HumanMessage(content="熊童子について教えて\
ください。")])
print(result.content)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

#1 プロンプトテンプレートの作成
message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

model = ChatOpenAI(model="gpt-4o-mini")
chain = prompt | model

question_text = " 熊童子について教えてください。"
information_text = """\
熊童子はベンケイソウ科コチレドン属の多肉植物です。
葉に丸みや厚みがあり、先端には爪のような突起があることから「熊の手」という\
愛称で人気を集めています。
花はオレンジ色のベル型の花を咲かせることがあります。"""

response = chain.invoke({"context": information_text, "question":
question_text})
print(response.content)

## 3.1.2 文書の構造化

In [ ]:
from langchain_core.documents import Document

#1 Documentクラスオブジェクトの作成
document = Document(
        page_content="""\
セダムはベンケイソウ科マンネングザ属で、日本にも自生しているポピュラーな多\
肉植物です。
種類が多くて葉の大きさや形状、カラーバリエーションも豊富なので、組み合わせ\
て寄せ植えにしたり、庭のグランドカバーにしたりして楽しむことができます。
とても丈夫で育てやすく、多肉植物を初めて育てる方にもおすすめです。""",
        metadata={"source": "succulent-plants-doc"},
    )

print(document)

In [ ]:
!pip install langchain_chroma

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

#1 Documentクラスオブジェクトの作成
documents = [
    Document(
        page_content="""\
セダムはベンケイソウ科マンネングザ属で、日本にも自生しているポピュラーな多\
肉植物です。
種類が多くて葉の大きさや形状、カラーバリエーションも豊富なので、組み合わせ\
て寄せ植えにしたり、庭のグランドカバーにしたりして楽しむことができます。
とても丈夫で育てやすく、多肉植物を初めて育てる方にもおすすめです。""",
        metadata={"source": "succulent-plants-doc"},
    ),
    Document(
        page_content="""\
熊童子はベンケイソウ科コチレドン属の多肉植物です。
葉に丸みや厚みがあり、先端には爪のような突起があることから「熊の手」という\
愛称で人気を集めています。
花はオレンジ色のベル型の花を咲かせることがあります。""",
        metadata={"source": "succulent-plants-doc"},
    ),
    Document(
        page_content="""\
エケベリアはベンケイソウ科エケベリア属の多肉植物で、メキシコなど中南米が原\
産です。
まるで花びらのように広がる肉厚な葉が特徴で、秋には紅葉も楽しめます。
品種が多く、室内でも気軽に育てられるので、人気のある多肉植物です。""",
        metadata={"source": "succulent-plants-doc"},
    ),
    Document(
        page_content="""\
ハオルチアは、春と秋に成長するロゼット形の多肉植物です。
密に重なった葉が放射状に展開し、幾何学的で整った株姿になるのが魅力です。
室内でも育てやすく手頃なサイズの多肉植物です。""",
        metadata={"source": "succulent-plants-doc"},
    ),
]

#2 Chromaデータベースの作成
vectorstore = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(),
)


In [ ]:
vectorstore.similarity_search("熊童子")

In [ ]:
vectorstore.similarity_search_with_score("熊童子")

## 3.1.3 文書検索機能を持つLLM

In [ ]:
from langchain_core.runnables import RunnableLambda

#1 Runnable オブジェクトの作成
retriever = RunnableLambda(vectorstore.similarity_search).bind\
(k=1)
retriever.invoke("熊童子")

In [ ]:
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])
model = ChatOpenAI(model="gpt-4o-mini")

#1 Chainの作成
rag_chain = {"context": retriever,
             "question": RunnablePassthrough()} | prompt | model

result = rag_chain.invoke("熊童子について教えてください。")
print(result.content)